In [20]:
pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [21]:
import duckdb
import os
from pathlib import Path
import pandas as pd
import numpy as np
from typing import List
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [22]:
# Configuration
DB_NAME = "amazing.duckdb"
data_folder = Path("./data")

con = duckdb.connect(DB_NAME)

In [23]:
def get_file_paths() -> List[str]:
    return sorted([str(p.resolve()) for p in data_folder.glob("*.csv")])

In [24]:
def load_new_files(con, file_paths: List[str]):
    print(f"Début du chargement de {len(file_paths)} fichier(s)...\n")
    for i, path in enumerate(file_paths, 1):
        filename = os.path.basename(path)

        print(f"[{i}/{len(file_paths)}] Vérification de {filename}...")
        already_loaded = con.execute(
            "SELECT 1 FROM loaded_files WHERE filename = ?", [filename]
        ).fetchone()

        if already_loaded:
            print(f"{filename} déjà chargé. Ignoré.\n")
            continue

        print(f"⬆Chargement de {filename} dans all_events...")
        con.execute(f"""
            INSERT INTO all_eventsp
            SELECT * FROM read_csv_auto('{path}', AUTO_DETECT=TRUE, SAMPLE_SIZE=-1)
        """)
        con.execute("INSERT INTO loaded_files VALUES (?)", [filename])
        print(f"{filename} ajouté avec succès à la base.\n")

    print("Chargement terminé.\n")

In [25]:
def init_loaded_table(con):
    print("Initialisation de la table 'loaded_files'...")
    con.execute("""
        CREATE TABLE IF NOT EXISTS loaded_files (
            filename TEXT PRIMARY KEY
        );
    """)
    print("Table 'loaded_files' prête.\n")

In [26]:
def create_all_events_table(con):
    con.execute("""
        CREATE TABLE IF NOT EXISTS all_events (
            event_time TIMESTAMP,
            event_type TEXT,
            product_id TEXT,
            category_id TEXT,
            category_code TEXT,
            brand TEXT,
            price DOUBLE,
            user_id TEXT,
            user_session TEXT
        );
    """)

In [27]:
def diagnostic(con):
    print(con.execute("SELECT COUNT(*) FROM all_events").fetchall())
    print(con.execute("SELECT * FROM all_events LIMIT 5").fetchdf())

In [28]:
def generate_user_features(con):
    print("Génération des features users...")
    con.execute("""
        CREATE OR REPLACE TABLE user_features AS
        SELECT
            user_id,
            COUNT(*) AS nb_events,
            COUNT(DISTINCT user_session) AS nb_sessions,

            -- Détail des types d'événements
            COUNT(CASE WHEN event_type = 'view' THEN 1 END) AS nb_view,
            COUNT(CASE WHEN event_type = 'cart' THEN 1 END) AS nb_cart,
            COUNT(CASE WHEN event_type = 'remove_from_cart' THEN 1 END) AS nb_remove,
            COUNT(CASE WHEN event_type = 'purchase' THEN 1 END) AS nb_achats,

            SUM(CASE WHEN event_type = 'purchase' THEN price ELSE 0 END) AS total_depense,
            AVG(CASE WHEN event_type = 'purchase' THEN price ELSE NULL END) AS avg_price,
            COUNT(DISTINCT category_code) AS nb_categories_visitees,
            MODE() WITHIN GROUP (ORDER BY category_code) AS category_top

        FROM (
            SELECT * FROM all_events
            WHERE user_id IS NOT NULL
            LIMIT 100000
        )
        GROUP BY user_id;
    """)
    print(" Table user_features créée")

In [29]:

# 2. Initialisation des tables AVANT d’y accéder
init_loaded_table(con)
create_all_events_table(con)

# 3. Chargement des fichiers
files = get_file_paths()
load_new_files(con, files)

create_all_events_table(con)
diagnostic(con)
generate_user_features(con)

Initialisation de la table 'loaded_files'...
Table 'loaded_files' prête.

Début du chargement de 7 fichier(s)...

[1/7] Vérification de 2019-Dec.csv...
2019-Dec.csv déjà chargé. Ignoré.

[2/7] Vérification de 2019-Nov.csv...
2019-Nov.csv déjà chargé. Ignoré.

[3/7] Vérification de 2019-Oct.csv...
2019-Oct.csv déjà chargé. Ignoré.

[4/7] Vérification de 2020-Apr.csv...
2020-Apr.csv déjà chargé. Ignoré.

[5/7] Vérification de 2020-Feb.csv...
2020-Feb.csv déjà chargé. Ignoré.

[6/7] Vérification de 2020-Jan.csv...
2020-Jan.csv déjà chargé. Ignoré.

[7/7] Vérification de 2020-Mar.csv...
2020-Mar.csv déjà chargé. Ignoré.

Chargement terminé.

[(411709736,)]
           event_time event_type product_id          category_id  \
0 2019-12-01 00:00:00       view    1005105  2232732093077520756   
1 2019-12-01 00:00:00       view   22700068  2232732091643068746   
2 2019-12-01 00:00:01       view    2402273  2232732100769874463   
3 2019-12-01 00:00:02   purchase   26400248  2053013553056579841   

In [30]:
nb_users = con.execute("SELECT COUNT(*) FROM all_events").fetchone()[0]

print(f"Taille de la table user_features : {nb_users} events")


Taille de la table user_features : 411709736 events


In [31]:
df_acheteurs = con.execute("""
    SELECT * FROM user_features WHERE nb_achats > 0
""").fetchdf()

df_curieux = con.execute("""
    SELECT * FROM user_features
    WHERE nb_view > 0 AND nb_achats = 0
""").fetchdf()


In [32]:
df_user_features = con.execute("SELECT * FROM user_features").fetchdf()
df_filtered = df_user_features.query("nb_events >= 5 and nb_achats > 0").copy()
features = df_filtered.select_dtypes(include=["number"]).fillna(0)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)

kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
df_filtered["cluster"] = kmeans.fit_predict(X_scaled)
df_filtered[["user_id", "cluster"]].head()


,user_id,cluster
1,539618099,3
35,512992433,3
63,514386085,0
68,569500893,3
83,512627296,1


In [33]:
con.execute("DESCRIBE user_features").fetchdf()

,column_name,column_type,null,key,default,extra
0,user_id,VARCHAR,YES,None,None,None
1,nb_events,BIGINT,YES,None,None,None
2,nb_sessions,BIGINT,YES,None,None,None
3,nb_view,BIGINT,YES,None,None,None
4,nb_cart,BIGINT,YES,None,None,None
5,nb_remove,BIGINT,YES,None,None,None
6,nb_achats,BIGINT,YES,None,None,None
7,total_depense,DOUBLE,YES,None,None,None
8,avg_price,DOUBLE,YES,None,None,None
9,nb_categories_visitees,BIGINT,YES,None,None,None


In [34]:
df_cat_visits = con.execute("""
    SELECT
        user_id,
        category_code,
        COUNT(*) AS nb_visites
    FROM all_events
    WHERE event_type = 'view'
      AND category_code IS NOT NULL
    GROUP BY user_id, category_code
    ORDER BY user_id, nb_visites DESC
""").fetchdf()

In [35]:
print("Aperçu des acheteurs :")
print(df_acheteurs.head())


Aperçu des acheteurs :
     user_id  nb_events  nb_sessions  nb_view  nb_cart  nb_remove  nb_achats  \
0  539618099         21            2       17        2          0          2   
1  512992433          9            1        7        1          0          1   
2  567157484          4            1        2        1          0          1   
3  514386085         15            2       13        1          0          1   
4  569500893          5            1        2        2          0          1   

   total_depense  avg_price  nb_categories_visitees  \
0         187.14      93.57                       2   
1          16.35      16.35                       2   
2         277.74     277.74                       1   
3         990.73     990.73                       2   
4         153.30     153.30                       2   

                        category_top  
0                 electronics.clocks  
1  appliances.environment.air_heater  
2     construction.components.faucet  
3        

In [36]:
print("Aperçu des visiteurs :")
print(df_curieux.head())

Aperçu des visiteurs :
     user_id  nb_events  nb_sessions  nb_view  nb_cart  nb_remove  nb_achats  \
0  525740700          3            1        3        0          0          0   
1  561181104         41            3       41        0          0          0   
2  569538767         13            1       13        0          0          0   
3  518831649          6            1        6        0          0          0   
4  579969800          2            1        2        0          0          0   

   total_depense  avg_price  nb_categories_visitees  \
0            0.0        NaN                       1   
1            0.0        NaN                      10   
2            0.0        NaN                       1   
3            0.0        NaN                       2   
4            0.0        NaN                       1   

                       category_top  
0          construction.tools.light  
1          construction.tools.light  
2          construction.tools.light  
3  appliances